In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
import joblib

np.set_printoptions(precision=4)

In [2]:
data = pd.read_csv('C:/ChuyenNganh_8/DSP_F1/f1-predictor/Document code/Dataset/final_df.csv')

In [3]:
data.tail()

,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,grid,podium,...,constructor_minardi,constructor_prost,constructor_red_bull,constructor_renault,constructor_sauber,constructor_team_lotus,constructor_toro_rosso,constructor_toyota,constructor_tyrrell,constructor_williams
16274,2023,22,True,False,False,False,False,ricciardo,15,11,...,False,False,False,False,False,False,False,False,False,False
16275,2023,22,True,False,False,False,False,sainz,16,18,...,False,False,False,False,False,False,False,False,False,False
16276,2023,22,True,False,False,False,False,kevin_magnussen,17,20,...,False,False,False,False,False,False,False,False,False,False
16277,2023,22,True,False,False,False,False,bottas,18,19,...,False,False,False,False,False,False,False,False,False,False
16278,2023,22,True,False,False,False,False,zhou,19,17,...,False,False,False,False,False,False,False,False,False,False


In [4]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < 2023]
X_train = train.drop(['driver', 'podium'], axis=1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

In [6]:
train.tail()

,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,grid,podium,...,constructor_minardi,constructor_prost,constructor_red_bull,constructor_renault,constructor_sauber,constructor_team_lotus,constructor_toro_rosso,constructor_toyota,constructor_tyrrell,constructor_williams
15853,2022,22,True,False,False,False,False,kevin_magnussen,16,0,...,False,False,False,False,False,False,False,False,False,False
15854,2022,22,True,False,False,False,False,gasly,17,0,...,False,False,False,False,False,False,False,False,False,False
15855,2022,22,True,False,False,False,False,bottas,18,0,...,False,False,False,False,False,False,False,False,False,False
15856,2022,22,True,False,False,False,False,albon,19,0,...,False,False,False,False,False,False,False,False,False,True
15857,2022,22,True,False,False,False,False,latifi,20,0,...,False,False,False,False,False,False,False,False,False,True


In [15]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == 2023]['round'].unique():

        test = df[(df.season == 2023) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop = True)
        prediction_df.sort_values('proba_1', ascending = False, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df[df.season == 2023]['round'].unique().max()
    return model_score

In [ ]:
params={'hidden_layer_sizes': [(80,20,40,5), (75,25,50,10)],
        'activation': ['identity', 'tanh', 'relu'],
        'solver': ['lbfgs', 'sgd', 'adam'],
        'alpha': np.logspace(-4,2,20)}


best_score = 0
best_model = None

for hidden_layer_sizes in params['hidden_layer_sizes']:
    for activation in params['activation']:
        for solver in params['solver']:
            for alpha in params['alpha']:
                model = MLPClassifier(
                    hidden_layer_sizes=hidden_layer_sizes,
                    activation=activation,
                    solver=solver,
                    alpha=alpha,
                    random_state=1
                )
                model.fit(X_train, y_train)

                model_score = score_classification(model)

                if model_score > best_score:
                    best_score = model_score
                    best_model = model

# Save the best model
if best_model:
    joblib.dump(best_model, 'nn_classifier_weight.pkl')
    print(f"Best model saved with score: {best_score}")
else:
    print("No model was saved.")

d:\env\envs\pytorch\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
d:\env\envs\pytorch\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\env\envs\pytorch\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\env\envs\pytorch\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maxim

Best model saved with score: 0.9545454545454546


In [ ]:
import joblib

nn_classifier = joblib.load('nn_classifier_weight.pkl')

print("Model parameters:")
print(nn_classifier.get_params())


Model parameters:
{'activation': 'tanh', 'alpha': 0.615848211066026, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (75, 25, 50, 10), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 1, 'shuffle': True, 'solver': 'sgd', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
